# 0. Initialisation et configuration

In [1]:
pip install linearmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.iv import IV2SLS

# --- CONFIGURATION (La seule partie à modifier si la base change) ---
CONFIG = {
    "file_name": "../données/OECD_debt_recent.csv",
    "mapping": {
        "Pays": "country",
        "Year": "year",
        "Nominal gross domestic product (billions)": "gdp",
        "Household debt, loans and debt securities (percent of GDP)": "debt_hh",
        "Non-financial corporations debt, loans and debt securities (percent of GDP)": "debt_corp",
        "General government debt (percent of GDP)": "debt_gov"
    },
    "target": "growth",       # Nom générique de la variable à prédire
    "interest_vars": ["debt_hh", "debt_corp", "debt_gov"] # Nos variables de dette
}

## Nettoyage

In [3]:

def load_and_prepare_data(config):
    # Chargement
    df = pd.read_csv(config["file_name"])
    
    # Renommage selon le mapping
    df = df.rename(columns=config["mapping"])
    
    # Filtrage : on enlève les colonnes non renseignées (all instruments)
    keep_cols = list(config["mapping"].values())
    df = df[keep_cols]
    
    # Nettoyage des types (Conversion string "12,5" -> float 12.5)
    for col in df.columns:
        if col not in ["country", "year"]:
            df[col] = df[col].astype(str).str.replace(',', '.').replace('nan', np.nan).astype(float)
    
    # Tri chronologique par pays
    df = df.sort_values(["country", "year"])
        
    return df

# Exécution
df_master = load_and_prepare_data(CONFIG)
print(f"Base de données prête : {df_master.shape[0]} lignes, {df_master.shape[1]} colonnes.")
print(df_master.head())

Base de données prête : 1102 lignes, 6 colonnes.
     country  year     gdp  debt_hh  debt_corp  debt_gov
0  Australia  1996  542.40    56.57      59.33     29.35
1  Australia  1997  573.07    59.98      61.28     25.91
2  Australia  1998  606.13    63.55      62.99     23.70
3  Australia  1999  638.38    67.93      63.90     22.54
4  Australia  2000  687.43    70.18      68.72     19.50


In [4]:
df_master.head()

,country,year,gdp,debt_hh,debt_corp,debt_gov
0,Australia,1996,542.40,56.57,59.33,29.35
1,Australia,1997,573.07,59.98,61.28,25.91
2,Australia,1998,606.13,63.55,62.99,23.70
3,Australia,1999,638.38,67.93,63.90,22.54
4,Australia,2000,687.43,70.18,68.72,19.50


In [5]:
df_master.isna().sum().sum()

np.int64(145)

Il y a 145 NaN dans notre data frame.

In [6]:
df_master.isna().sum(axis=1).value_counts().sort_index()


0    1001
1      66
2      26
3       9
Name: count, dtype: int64

1001 lignes n'ont pas de NaN sur les 1101 lignes de notre data frame. Cela représente 91% du data frame. 

In [7]:
df_master.head(100)

,country,year,gdp,debt_hh,debt_corp,debt_gov
0,Australia,1996,542.40,56.57,59.33,29.35
1,Australia,1997,573.07,59.98,61.28,25.91
2,Australia,1998,606.13,63.55,62.99,23.70
3,Australia,1999,638.38,67.93,63.90,22.54
4,Australia,2000,687.43,70.18,68.72,19.50
...,...,...,...,...,...,...
95,Canada,2004,1335.73,69.61,84.13,71.89
96,Canada,2005,1421.59,72.26,77.42,70.64
97,Canada,2006,1496.60,75.90,78.26,69.92
98,Canada,2007,1577.66,80.73,81.12,67.18


Pour remplir les Nan, on va utiliser la méthode d'interpolation qui remplit les NaN par une estimation linéaire de ce que devrait être la valeur étant donné son passé et son future. 

In [8]:
cols_to_interp = ["gdp", "debt_hh", "debt_corp", "debt_gov"]

df_master = df_master.sort_values(["country", "year"])

df_master[cols_to_interp] = (
    df_master
        .groupby("country")[cols_to_interp]
        .transform(lambda x: x.interpolate(limit_direction="both"))
)


In [9]:
df_master.isna().sum().sum()

np.int64(58)

L'interpolation nous à permis de réduire le nombre de Nan à 58.

In [10]:
# Création des Lags (Dette à t-1)
interest_vars = ["debt_hh", "debt_corp", "debt_gov"]

for var in interest_vars:
    df_master[f"{var}_lag1"] = df_master.groupby("country")[var].shift(1)


# Taux de croissance du PIB : ln(PIB_t) - ln(PIB_t-1)
df_master["growth"] = df_master.groupby("country")["gdp"].apply(lambda x: np.log(x).diff() * 100).reset_index(level=0, drop=True)

In [11]:
df_master.head(20)

,country,year,gdp,debt_hh,debt_corp,debt_gov,debt_hh_lag1,debt_corp_lag1,debt_gov_lag1,growth
0,Australia,1996,542.40,56.57,59.33,29.35,NaN,NaN,NaN,NaN
1,Australia,1997,573.07,59.98,61.28,25.91,56.57,59.33,29.35,5.500414
2,Australia,1998,606.13,63.55,62.99,23.70,59.98,61.28,25.91,5.608661
3,Australia,1999,638.38,67.93,63.90,22.54,63.55,62.99,23.70,5.183923
4,Australia,2000,687.43,70.18,68.72,19.50,67.93,63.90,22.54,7.402629
5,Australia,2001,730.49,73.88,65.36,17.11,70.18,68.72,19.50,6.075554
6,Australia,2002,782.37,81.39,63.69,15.01,73.88,65.36,17.11,6.861223
7,Australia,2003,830.19,90.62,62.04,13.19,81.39,63.69,15.01,5.932682
8,Australia,2004,894.33,96.79,61.93,11.91,90.62,62.04,13.19,7.442024
9,Australia,2005,964.21,101.33,67.78,10.86,96.79,61.93,11.91,7.523428


In [12]:
df_master.to_csv("dette_master.csv", index=False, encoding="utf-8-sig")

# Premières régressions

## OLS naïf

In [13]:
# On retire les lignes vides (dues au calcul des lags et de la croissance)
df_model = df_master.dropna(subset=["growth"] + [v+"_lag1" for v in CONFIG["interest_vars"]])

# Définition de la formule de manière dynamique
# Elle ressemblera à : "growth ~ debt_hh_lag1 + debt_corp_lag1 + debt_gov_lag1"
formula = "growth ~ " + " + ".join([f"{v}_lag1" for v in CONFIG["interest_vars"]])

print(f"Lancement du modèle : {formula}")

# Modèle OLS simple (Pooling)
model_ols = smf.ols(formula, data=df_model).fit()

# Affichage des résultats
print(model_ols.summary())

Lancement du modèle : growth ~ debt_hh_lag1 + debt_corp_lag1 + debt_gov_lag1
                            OLS Regression Results                            
Dep. Variable:                 growth   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     52.54
Date:                Tue, 17 Feb 2026   Prob (F-statistic):           1.51e-31
Time:                        08:27:55   Log-Likelihood:                -3305.3
No. Observations:                1008   AIC:                             6619.
Df Residuals:                    1004   BIC:                             6638.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

Ce modèle montre que la dette des ménages et celle de l'État pèsent significativement sur la croissance future, avec un impact négatif plus marqué pour les ménages ($-0,05$), tandis que la dette des entreprises n'a pas d'effet statistiquement prouvé ($p=0,44$). Malgré un $R^2$ de $12,8\%$, la faible valeur de Durbin-Watson ($1,09$) révèle une autocorrélation des résidus, indiquant que le modèle ignore des spécificités nationales ou des chocs temporels que les effets fixes et le modèle CS-ARDL devront traiter.

### Introduction des effets fixes

In [14]:
# Préparation des données pour le Panel (Index double : Pays puis Année)
df_panel = df_model.set_index(['country', 'year'])

# Modèle avec Effets Fixes par Pays (Entity Effects)
# On utilise les mêmes variables mais on ajoute "EntityEffects"
exog_vars = [f"{v}_lag1" for v in CONFIG["interest_vars"]]
exog = sm.add_constant(df_panel[exog_vars])

model_fe = PanelOLS(df_panel['growth'], exog, entity_effects=True)
results_fe = model_fe.fit()

print(results_fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 growth   R-squared:                        0.0554
Estimator:                   PanelOLS   R-squared (Between):              0.0415
No. Observations:                1008   R-squared (Within):               0.0554
Date:                Tue, Feb 17 2026   R-squared (Overall):              0.0501
Time:                        08:27:55   Log-likelihood                   -3109.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      18.934
Entities:                          36   P-value                           0.0000
Avg Obs:                       28.000   Distribution:                   F(3,969)
Min Obs:                       28.000                                           
Max Obs:                       28.000   F-statistic (robust):             18.934
                            

Formellement, le modèle devient :

$$
growth_{it} = \alpha_i + \beta_1 \, debt\_hh\_lag1_{it} + \beta_2 \, debt\_corp\_lag1_{it} + \dots + \varepsilon_{it}
$$

où :

- $\alpha_i$ = **effet fixe du pays $i$**  
- Les coefficients $\beta$ sont estimés en utilisant la **variation dans le temps au sein de chaque pays**  
- Les différences **entre pays** sont absorbées par $\alpha_i$



L'introduction des effets fixes renforce l'impact négatif de la dette des ménages, dont le coefficient chute de $-0,05$ à $-0,09$ ($p=0,000$). Ce modèle explique $6\%$ de la variance intra-pays ($R^2$ Within), prouvant que les caractéristiques structurelles propres à chaque nation masquaient une partie de l'effet néfaste de l'endettement sur la croissance. La forte significativité des effets d'entité ($p=0,000$) valide cette approche par rapport à l'OLS simple et prépare le terrain pour le modèle CS-ARDL.

## Variables instrumentales

Afin de traiter un potentiel biais d'endogénéité nous utilisons la méthode des doubles moindres carrés (2SLS). Nous instrumentons la dette des ménages par son retard à deux périodes ($t-2$), partant du principe que le niveau d'endettement passé influence la croissance actuelle mais n'est pas directement impacté par les chocs économiques de l'année en cours. Cette approche permet d'isoler la variation de la dette qui est purement exogène afin de confirmer la direction du lien de causalité.

In [15]:
# 1. Création des instruments (Lag 2)
# On utilise la dette d'il y a 2 ans pour instrumenter celle d'il y a 1 an
for var in CONFIG["interest_vars"]:
    df_panel[f'{var}_lag2'] = df_panel.groupby('country')[var].shift(2)

# 2. Nettoyage des valeurs manquantes créées par le second retard
df_iv = df_panel.dropna(subset=[f'{v}_lag2' for v in CONFIG["interest_vars"]] + ['growth'])

# 3. Définition du modèle IV
# Formule : Y ~ 1 + [Variables_Endogènes ~ Instruments]
# Ici, on instrumente la dette des ménages par son retard d'ordre 2
formula_iv = 'growth ~ 1 + [debt_hh_lag1 ~ debt_hh_lag2]'

model_iv = IV2SLS.from_formula(formula_iv, data=df_iv)
results_iv = model_iv.fit(cov_type='robust')

print(results_iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 growth   R-squared:                      0.0921
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0911
No. Observations:                 936   F-statistic:                    78.023
Date:                Tue, Feb 17 2026   P-value (F-stat)                0.0000
Time:                        08:27:55   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept        9.2480     0.5196     17.797     0.0000      8.2296      10.266
debt_hh_lag1    -0.0631     0.0071    -8.833

L'estimation par variables instrumentales confirme la robustesse de l'impact négatif de l'endettement des ménages sur la croissance. Le coefficient de $-0,0504$ ($p=0,000$) est très proche de celui obtenu avec le modèle OLS initial ($-0,0532$), ce qui suggère que le biais d'endogénéité était limité ou que le lien causal de la dette vers la croissance est particulièrement stable. Bien que le $R^2$ soit plus faible ($6,8\%$), ce qui est classique en approche IV car seule la fraction "exogène" de la variance est conservée, la forte significativité statistique valide l'idée que l'accumulation de dette des ménages agit comme un frein structurel à l'activité économique.

# Modèle CS-ARDL

Le modèle CS-ARDL constitue une extension dynamique du cadre de régression précédent. Il permet de répondre aux trois principaux problèmes identifiés dans la littérature empirique sur le lien entre dette et croissance, notamment par Lombardi (2012).

Premièrement, il introduit une dynamique temporelle via la structure Autoregressive Distributed Lag (ARDL). Contrairement à une régression statique, ce modèle inclut un retard de la variable dépendante ($y_{t-1}$) ainsi que des retards des variables explicatives ($x_{t-1}$). Cette spécification permet de capturer les mécanismes d’ajustement progressifs : l’effet de l’endettement sur la croissance ne se matérialise pas instantanément, mais se diffuse dans le temps.

Deuxièmement, l’approche ARDL permet de réduire les problèmes de simultanéité. En se fondant sur la chronologie des variables, le modèle exploite le fait que les niveaux d’endettement passés peuvent influencer la croissance future, alors que la croissance future ne peut pas affecter la dette passée. Cette structure dynamique limite ainsi les biais d’endogénéité présents dans les modèles statiques.

Troisièmement, le modèle corrige la dépendance transversale entre pays. Dans un panel d’économies de l’OCDE, les cycles conjoncturels sont fortement synchronisés. Des chocs globaux, tels que la crise financière de 2008, peuvent affecter simultanément la croissance de tous les pays. Sans correction, ces effets communs risquent d’être attribués à tort aux variables nationales d’endettement. Le modèle CS-ARDL introduit donc les moyennes transversales des variables (notées $\bar{Z}_t$), qui capturent ces facteurs globaux non observés.

La spécification estimée s’écrit :

$$y_{it} = \alpha_i + \sum_{j=1}^p \phi_j y_{i,t-j} + \sum_{j=0}^q \beta_j x_{i,t-j} + \sum_{j=0}^k \gamma_j \bar{y}_{t-j} + \sum_{j=0}^m \delta_j \bar{x}_{t-j} + \epsilon_{it}$$

où :

- $y_{i,t-j}$ est la croissance du pays $i$ à la date $t-j$,

- $x_{i,t-j}$ représente le vecteur des variables d’endettement nationales,

- $\bar{Z}_{t}$ correspond aux moyennes transversales des variables,

- $\alpha_i$ capte les caractéristiques structurelles propres à chaque pays.

- p, q, k et m le nombre de retards choisi

Cette spécification permet ainsi d’identifier un effet de la dette sur la croissance en tenant compte simultanément des dynamiques internes aux pays et des chocs macroéconomiques communs.

In [16]:
from linearmodels.panel import PanelOLS

def add_lags(df, group_col, vars_list, max_lag, suffix="lag"):
    out = df.copy()
    for v in vars_list:
        for L in range(1, max_lag+1):
            out[f"{v}_{suffix}{L}"] = out.groupby(group_col)[v].shift(L)
    return out

def add_cs_means(df, time_col, vars_list, lags_list=None, prefix="cs_"):
    out = df.copy()
    cols = vars_list.copy()
    if lags_list:
        cols += lags_list
    for c in cols:
        out[f"{prefix}{c}"] = out.groupby(time_col)[c].transform("mean")
    return out

# -------------------------
# PARAMS CS-ARDL
# -------------------------
q = 1  # nb de retards sur les dettes (à tester: 1,2,3)
p = 1  # nb de retards sur y (ici 1)

df_cs = df_master.copy()

# y = growth
# x = dettes en niveau (% GDP) plutôt que seulement lag1 déjà pré-calculé
x_vars = CONFIG["interest_vars"]  # ['debt_hh','debt_corp','debt_gov']

# Lags
df_cs = add_lags(df_cs, "country", ["growth"] + x_vars, max_lag=max(p, q), suffix="lag")

include_contemporaneous_x = True  # inclure les xit dans le modèle 

# Construire la liste des régressseurs
regressors = []
# AR part
regressors += [f"growth_lag{L}" for L in range(1, p+1)]

# DL part
if include_contemporaneous_x:
    regressors += x_vars
regressors += [f"{v}_lag{L}" for v in x_vars for L in range(1, q+1)]

# Ajouter CS means (contemporains + lags qui correspondent à ce que tu mets dans le modèle)
cs_source_cols = regressors  # on prend les mêmes colonnes → spec “symétrique”
df_cs = add_cs_means(df_cs, "year", vars_list=cs_source_cols, lags_list=None, prefix="cs_")

cs_regressors = [f"cs_{c}" for c in cs_source_cols]

# Dataset final
needed = ["country","year","growth"] + regressors + cs_regressors
df_cs = df_cs[needed].dropna().copy()
df_cs = df_cs.set_index(["country","year"])

# Formule PanelOLS
rhs = " + ".join(regressors + cs_regressors)
formula_cs_ardl = f"growth ~ {rhs} + EntityEffects"

model = PanelOLS.from_formula(formula_cs_ardl, data=df_cs, drop_absorbed=True)

res = model.fit(cov_type="clustered", cluster_entity=True)
print(res.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 growth   R-squared:                        0.5954
Estimator:                   PanelOLS   R-squared (Between):             -15.389
No. Observations:                 972   R-squared (Within):               0.5954
Date:                Tue, Feb 17 2026   R-squared (Overall):             -9.6962
Time:                        08:27:55   Log-likelihood                   -2557.6
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      96.914
Entities:                          36   P-value                           0.0000
Avg Obs:                       27.000   Distribution:                  F(14,922)
Min Obs:                       27.000                                           
Max Obs:                       27.000   F-statistic (robust):             60.789
                            

### Estimation des effets à long terme et à court terme

In [17]:
# 1. Extraction des coefficients
params = res.params

# 2. Calcul pour chaque variable de dette (x_vars)
lt_effects = {}
st_effects = {}

# On récupère le coefficient du retard de la variable dépendante (phi)
# Dans ton code p=1, donc c'est 'growth_lag1'
phi = params['growth_lag1']

for var in x_vars:
    # Effet Court Terme (CT) : coefficient de x_it
    beta_0 = params[var]
    st_effects[var] = beta_0
    
    # Effet Long Terme (LT) : (beta_0 + beta_1 + ...) / (1 - phi)
    # On récupère tous les retards de cette variable x (ici q=1)
    beta_lags = [params[f"{var}_lag{L}"] for L in range(1, q+1)]
    
    numerator = beta_0 + sum(beta_lags)
    denominator = 1 - phi
    
    lt_effects[var] = numerator / denominator

# 3. Affichage propre
results_df = pd.DataFrame({
    'Short-Term Effect': st_effects,
    'Long-Term Effect': lt_effects
})

print("--- Effets de la Dette sur la Croissance ---")
print(results_df)

--- Effets de la Dette sur la Croissance ---
           Short-Term Effect  Long-Term Effect
debt_hh            -0.325218          0.004188
debt_corp           0.001073          0.001639
debt_gov           -0.177492         -0.037936


## Implémentation des effets de seuil

Dans la régression, au lieu d'avoir un seul $\beta \cdot x_{it}$, il y a désormais :$$\beta_{low} \cdot x_{it}^{low} + \beta_{high} \cdot x_{it}^{high} $$
Où :
- $x_{it}^{low}$ vaut la valeur de la dette si elle est $\le$ au seuil, et $0$ sinon.
- $x_{it}^{high}$ vaut la valeur de la dette si elle est $>$ au seuil, et $0$ sinon.

Cela permet en plus des effets de seuil d'analyser la variation de la croissance suite à une augmentation de la dette selon que l'on dépasse ou non le seuil.

In [18]:
# Définition des seuils (exemples en % du PIB)
thresholds = {
    'debt_hh': 60,
    'debt_corp': 90,
    'debt_gov': 85
}

df_threshold = df_master.copy()

# Création des variables conditionnelles
# On crée 'debt_var_low' qui vaut la valeur de la dette si < seuil, sinon 0
for var, limit in thresholds.items():
    # Dummy : 1 si sous le seuil, 0 si au-dessus
    dummy_name = f"dummy_low_{var}"
    df_threshold[dummy_name] = (df_threshold[var] <= limit).astype(int)
    
    # Interaction : on garde la valeur de la dette uniquement si on est sous le seuil
    # Cela permet de tester si la pente change selon le régime
    df_threshold[f"{var}_low"] = df_threshold[var] * df_threshold[dummy_name]
    df_threshold[f"{var}_high"] = df_threshold[var] * (1 - df_threshold[dummy_name])

# Mise à jour des variables d'intérêt pour la régression
# Ici, on remplace les variables de dette par leurs versions segmentées
segmented_vars = []
for var in x_vars:
    segmented_vars.extend([f"{var}_low", f"{var}_high"])

# -------------------------
# RELANCER LA LOGIQUE CS-ARDL
# -------------------------
p, q = 1, 1

# Lags sur la croissance et les nouvelles variables segmentées
df_cs = add_lags(df_threshold, "country", ["growth"] + segmented_vars, max_lag=max(p, q))

# Construction des régresseurs (Individuels + CS Means)
regressors = [f"growth_lag1"] + segmented_vars
regressors += [f"{v}_lag1" for v in segmented_vars]

# Ajout des CS Means sur les versions segmentées
df_cs = add_cs_means(df_cs, "year", vars_list=regressors, prefix="cs_")
cs_regressors = [f"cs_{c}" for c in regressors]

# Finalisation du dataset
needed = ["country","year","growth"] + regressors + cs_regressors
df_cs = df_cs[needed].dropna().set_index(["country","year"])

# Estimation
rhs = " + ".join(regressors + cs_regressors)
model_threshold = PanelOLS.from_formula(f"growth ~ {rhs} + EntityEffects", data=df_cs)
res_threshold = model_threshold.fit(cov_type="clustered", cluster_entity=True)

print(res_threshold.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 growth   R-squared:                        0.6315
Estimator:                   PanelOLS   R-squared (Between):             -14.683
No. Observations:                 972   R-squared (Within):               0.6315
Date:                Tue, Feb 17 2026   R-squared (Overall):             -9.2284
Time:                        08:27:56   Log-likelihood                   -2512.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      59.987
Entities:                          36   P-value                           0.0000
Avg Obs:                       27.000   Distribution:                  F(26,910)
Min Obs:                       27.000                                           
Max Obs:                       27.000   F-statistic (robust):             137.40
                            